# 1.load package

In [ ]:
library(imcRtools)
library(cytomapper)
library(mclust)
library(scuttle)
library(tidyverse)
library(ggrepel)
library(EBImage)
library(dittoSeq)
library(viridis)
library(Rphenograph)
library(igraph)
library(dittoSeq)
library(viridis)
library(scater)
library(tableone)
library(openxlsx)
library(gridExtra)

# 2.creat spe subject

In [ ]:
# read in raw data
spe <- read_steinbock("P:/PI/PI_Chuva_de_Sousa_Lopes/susana/Fu/Project/4-CyTOF immune cell/experiment/1-xueying/Analysis_R/2st_for paper_frozen_2025_1_27_revised segmentation/")
spe

In [ ]:
# read in mask and image
masks <- loadImages("masks/", as.is = TRUE)
images <- loadImages("img/")

In [ ]:
# process data
colnames(spe) <- paste0(spe$sample_id, "_", spe$ObjectNumber)
channelNames(images) <- rownames(spe)
all.equal(names(images), names(masks))

# Store patient and image level information in elementMetadata
mcols(images) <- mcols(masks) <- DataFrame(sample_id = names(images))
colnames(spe) <- paste0(spe$sample_id, "_", spe$ObjectNumber)

In [ ]:
genes_of_interest <- c("CD45", "CD14", "FOXp3", "FcεRIα", "CD69", "CD4", "CD8a", 
                       "CollagenI", "CD31", "ECadherin", "CD123", "CD141", "CD7", 
                       "CD163", "CD103", "CD127", "CD30", "CD68", "CD11b", "CD20", 
                       "CD11c", "CD15", "CD161", "CD117", "Ki-67", "CD27", "HLA-DR", 
                       "CD45RA", "CD3", "CD1c", "CD38", "CD45RO", "CD57", "CD25", 
                       "CD56", "SMA")
rowData(spe)$use_channel <- rownames(spe) %in% genes_of_interest

In [ ]:
## tranform  raw data by asinh
assay(spe, "exprs_0.5") <- asinh(counts(spe)/0.5)

# 3.downstream analysis of whole sample

## 3.1 tsne

In [ ]:
set.seed(220224)
spe <- runUMAP(spe, subset_row = rowData(spe)$use_channel, exprs_values = "exprs_0.5") 
spe <- runTSNE(spe, subset_row = rowData(spe)$use_channel, exprs_values = "exprs_0.5") 

mat <- t(assay(spe, "exprs_0.5")[rowData(spe)$use_channel,])

set.seed(230619)

      
out1 <- Rphenograph(mat, k = 50)

clusters1 <- factor(membership(out1[[2]]))

spe$pg_clusters_exprs0.05<- clusters1


In [ ]:
colorcode <- c(
  "#FF5733", "#33FF57", "#5733FF", "#FF33A5", "#33FFF0",
  "#A533FF", "#33A5FF", "#FF5733", "#FFC300", "#DAF7A6",
  "#C70039", "#900C3F", "#581845", "#1ABC9C", "#2ECC71",
  "#3498DB", "#9B59B6", "#34495E", "#16A085", "#27AE60",
  "#2980B9", "#8E44AD", "#2C3E50", "#F1C40F", "#E67E22",
  "#E74C3C", "#ECF0F1", "#95A5A6", "#F39C12", "#D35400",
  "#C0392B"
)

In [ ]:
options(repr.plot.width=12, repr.plot.height=10)
p=dittoDimPlot(spe, var = "pg_clusters1", 
             reduction.use = "TSNE", size =1,
             do.label =FALSE) +scale_color_manual(values = metadata(spe)$color_vectors$cluster)
    ggtitle("Phenograph clusters on TSNE")
ggsave(
  "TSNE_clusters.pdf",
  plot = p,
  width = 12,
  height = 10,
  units = "in",
  dpi = 300
)

## 3.2 heatmap

In [ ]:
library(ComplexHeatmap)
library(SingleCellExperiment)
library(circlize) 

In [ ]:
celltype_mean <- aggregateAcrossCells(as(spe, "SingleCellExperiment"),  
                     ids = spe$pg_clusters1, 
                     statistics = "mean",
                     use.assay.type = "exprs_0.5", 
                     subset.row = rownames(spe)[rowData(spe)$use_channel])
heatmap_data <- assay(celltype_mean, "exprs_0.5")
head(heatmap_data)

In [ ]:
color_dict <- c(
  "1" = "#FF5733", 
  "2" = "#33FF57", 
  "3" = "#5733FF", 
  "4" = "#FF33A5", 
  "5" = "#33FFF0", 
  "6" = "#A533FF", 
  "7" = "#33A5FF", 
  "8" = "#FF5733", 
  "9" = "#FFC300", 
  "10" = "#DAF7A6", 
  "11" = "#C70039", 
  "12" = "#900C3F", 
  "13" = "#581845", 
  "14" = "#1ABC9C", 
  "15" = "#2ECC71", 
  "16" = "#3498DB", 
  "17" = "#9B59B6", 
  "18" = "#34495E", 
  "19" = "#16A085", 
  "20" = "#27AE60", 
  "21" = "#2980B9", 
  "22" = "#8E44AD", 
  "23" = "#2C3E50", 
  "24" = "#F1C40F", 
  "25" = "#E67E22"
)

group <- colnames(heatmap_data)


col_annot <- HeatmapAnnotation(
  Group = group,
  col = list(Group = color_dict)
)

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
pdf("heatmap_spe_CD45_ComplexHeatmap_zscore_manhattan.pdf", width = 10, height = 10)
z_score_matrix <- t(scale(t(heatmap_data), center = TRUE, scale = TRUE))
col_fun <- colorRamp2(c(-2, 0, 2), c("blue", "white", "red"))
Heatmap(z_score_matrix,
        name = "Z-score",
        col = col_fun,
        top_annotation = col_annot,
        show_row_names = TRUE,
        show_column_names = TRUE,
        clustering_distance_rows = "euclidean",
        clustering_distance_columns = "manhattan",  #manhattan   euclidean
        #clustering_method_rows = FALSE,
        clustering_method_columns = "complete",
        row_title = "Genes",
        column_title = "Sample Groups")
dev.off()

## 3.3 mapping of cluster

In [ ]:
cluster_mapping <- c(
  "1" = 1, "11" = 1, "18" = 1, "13" = 1, "22" = 1,"16" = 1,
  "20" = 2,
  "7" = 3,
  "12" = 4,
  "3" = 5, "4" = 5, "14" = 5, "6" = 5,
  "17" = 6,
  "9" = 7, "10" = 7,
  "15" = 8,
  "2" = 9,
  "5" = 10,
  "23" = 11,
  "21" = 12,
  "24" = 13,
  "25" = 14,
  "8" = 15, "19" = 15
)


colData(spe)$cell_type <- cluster_mapping[as.character(colData(spe)$pg_clusters1)]
spe$cell_type <- as.character(spe$cell_type)

In [ ]:
color_mapping <- c(
  "1"  = "#F032E6",  
  "2"  = "#FEEC37",
  "3"  = "#3CB44B", 
  "4"  = "#E6194B", 
  "5"  = "#756bb1",  
  "6"  = "#d9d9d9",  
  "7"  = "#646464",  
  "8"  = "#F58231",  
  "9"  = "#4D4D4D", 
  "10" = "#344cb7", 
  "11" = "#008080", 
  "12" = "#FF1493",  
  "13" = "#00FFFF",  
  "14" = "#AA6E28",  
  "15" = "#7FFF00"  
)

In [ ]:
plotCells(masks,
          object = spe, 
          cell_id = "ObjectNumber", 
          img_id = "sample_id",
          colour_by = "cell_type",
          colour = list(cell_type = color_mapping),
          missing_colour = "gray",
         display = "single",
         scale_bar=NULL,
          image_title=NULL,
          save_plot = list(filename = "whole_sample_mapping.png", scale = 1))

## 3.4 visulization of follicle type

In [ ]:
a=colData(spe)
a=as.data.frame(a)
b=spatialCoords(spe)
b=as.data.frame(b)
a$Pos_X=b$Pos_X
a$Pos_Y=b$Pos_Y

In [ ]:
library(sf)
library(dplyr)

type2b_1 <- subset(a, sample_id == "type3b_3")
type2b_1$cell_id=rownames(type2b_1)



polygon_wkt <- "POLYGON ((7 242, 0 992, 699 992, 1000 801, 851 733, 617 708, 241 432, 7 242))"


polygon_sf <- st_as_sfc(polygon_wkt) %>% 
  st_set_crs(NA)  


points_sf <- st_as_sf(type2b_1, 
                      coords = c("Pos_X", "Pos_Y"), 
                      crs = NA)  


type2b_1$inside <- st_intersects(points_sf, polygon_sf, sparse = FALSE)[, 1]


result <- type2b_1 %>% 
  filter(inside == TRUE) %>% 
  select(-inside) 


result$cell_id=rownames(result)


In [ ]:
library(dplyr)
meta <- meta %>%
  mutate(
    follicle = ifelse(
      cell_id %in% result$cell_id & follicle == 1, 
      "type3b_4",                                   
      follicle                                      
    )
  )

In [ ]:
meta <- meta %>%
  mutate(
    follicle_type = case_when( 
      follicle == "1" ~ "1",
      follicle == "CA" ~ "CA",
      
  
      TRUE ~ str_replace(follicle, "_\\d+$", "")  
    )
  )
table(meta$follicle_type)

In [ ]:
spe$follicle=meta$follicle
spe$follicle_type=meta$follicle_type

In [ ]:
color_mapping <- c(
  "1" = "gray",         
  "CA" = "green",        
  "cortex_in" = "red",  
  "cortex_out" = "blue", 
  "healthy" = "purple",  
  "type1" = "orange",    
  "type2a" = "cyan",    
  "type2b" = "magenta",  
  "type3b" = "yellow"   
)

In [ ]:
plotCells(
  masks,
  object = spe, 
  cell_id = "ObjectNumber", 
  img_id = "sample_id",
  colour_by = "follicle_type",
  colour = list(follicle_type = color_mapping),
  missing_colour = "gray",  display = "single",
         scale_bar=NULL,
          image_title=NULL,
          save_plot = list(filename = "ROI/ROI.png", scale = 1))


In [ ]:
##quantification

library(ggplot2)
library(dplyr)


plot_data <- meta_follicle %>%
  mutate(
    cell_type = factor(cell_type, levels = as.character(1:15)) 
  ) %>%
  group_by(follicle_type, cell_type) %>%
  summarise(count = n(), .groups = "drop") %>%
  group_by(follicle_type) %>%
  mutate(percentage = count / sum(count) * 100)


color_mapping <- c(
  "1"  = "#F032E6",  "2"  = "#FEEC37",  "3"  = "#3CB44B",
  "4"  = "#E6194B",  "5"  = "#756bb1",  "6"  = "#d9d9d9",
  "7"  = "#646464",  "8"  = "#F58231",  "9"  = "#4D4D4D",
  "10" = "#344cb7",  "11" = "#008080",  "12" = "#FF1493",
  "13" = "#00FFFF",  "14" = "#AA6E28",  "15" = "#7FFF00"
)


p <- ggplot(plot_data, aes(x = follicle_type, y = percentage, fill = cell_type)) +
  geom_bar(stat = "identity", position = "fill") +
  scale_y_continuous(
    labels = scales::percent_format(),
    expand = expansion(mult = c(0, 0.05)) 
  ) +
  scale_fill_manual(
    values = color_mapping,
    name = "Cell Type",
    drop = FALSE 
  ) +
  labs(
    x = "Follicle Type",
    y = "Percentage",
    title = "Cell Type Composition by Follicle Type"
  ) +
  theme_minimal(base_size = 14) +  
  theme(
    axis.text.x = element_text(
      angle = 45,
      hjust = 1,
      face = "bold", 
      color = "black"
    ),
    axis.text.y = element_text(
      face = "bold", 
      color = "black"
    ),
    axis.title.x = element_text(
      face = "bold", 
      size = 12,
      margin = margin(t = 10)
    ),
    axis.title.y = element_text(
      face = "bold",
      size = 12,
      margin = margin(r = 10)
    ),
    legend.title = element_text(face = "bold"),  
    plot.title = element_text(
      hjust = 0.5,
      face = "bold",
      size = 16
    ),
    panel.grid.major.x = element_blank(), 
    panel.grid.major.y = element_blank(),
    panel.border = element_blank(),  
    axis.line = element_line(size = 1.2, color = "black")  
      
  )


ggsave(
  filename = "cell_type_composition.pdf",
  plot = p,
  device = "pdf",
  width = 10,    
  height = 8,
  units = "in",
  dpi = 300
)
print(p)

# 4.downstream analysis of immune cell

## 4.1 visulization of CD45 positive cell

In [ ]:
data_counts$CD45_1 <- ifelse(data_counts$CD45 >1.5, 1, 2)

In [ ]:
# 1. 添加 immune_filter 列，默认值为 2
colData(spe)$immune_filter <- 2

# 2. 设置条件：CD45_1 == 1 且 pg_clusters1 在指定值中
# 定义目标 pg_clusters1 值
target_clusters <- c(1, 11, 18, 13, 22, 20, 16)

# 根据条件更新 immune_filter
spe$immune_filter[colData(spe)$CD45_1 == 1 & 
                  colData(spe)$pg_clusters1 %in% target_clusters] <- 1

In [ ]:
plotPixels(image =images,
           mask =masks,
           object = spe, 
           cell_id = "ObjectNumber", 
           img_id = "sample_id",
           colour_by =c("CD45"),
           outline_by = "immune_filter",
           colour = list(immune_filter= c("1"="#FFFFFF","2"="#45095C")),
            bcg = list(CD45 = c(0, 8, 1.2)),
          display = "single",
           scale_bar=NULL,
           image_title=NULL,
           image_title=NULL,
           thick = TRUE,
           save_plot = list(filename = "Figure_revised/9st/CD45_CD45_1.png", scale = 3))

In [ ]:
spe_cd45_9st=spe[,spe$immune_filter %in% c("1")]

## 4.2 tsne

In [ ]:
#alway use
genes_of_interest <- c("CD45", "CD14",  "FcεRIα", "CD69", "CD4", "CD8a",  "CD7", "CD163", "CD103", "CD127",
                      "CD68", "CD11b", "CD11c", "CD15", "CD161", "CD117", "CD27", "HLA-DR", "CD45RA", "CD3", "CD1c", "CD38")
rowData(spe_cd45_9st)$use_channel <- rownames(spe_cd45_9st) %in% genes_of_interest

In [ ]:
set.seed(230618)
spe_cd45_9st <- runUMAP(spe_cd45_9st, subset_row = rowData(spe_cd45_9st)$use_channel, exprs_values = "exprs_0.5") 
spe_cd45_9st <- runTSNE(spe_cd45_9st, subset_row = rowData(spe_cd45_9st)$use_channel, exprs_values = "exprs_0.5")

In [ ]:
mat <- t(assay(spe_cd45_9st, "exprs_0.5")[rowData(spe_cd45_9st)$use_channel,])

set.seed(230619)

      
out2 <- Rphenograph(mat, k = 49)

clusters1 <- factor(membership(out2[[2]]))
spe_cd45_9st$immune_9st1=clusters1

In [ ]:
spe_cd45_9st$immune_9st<- as.numeric(levels(spe_cd45_9st$immune_9st)[spe_cd45_9st$immune_9st]) - 1
spe_cd45_9st$immune_9st <- factor(spe_cd45_9st$immune_9st,
                                  levels = as.character(0:12))

In [ ]:
library(RColorBrewer)

colorcode <- c(
  "#C1A029",
  "#ff002a",
  "#009203",
  "#FEEC37",
  "#4fa9ff",
  "#00FFFF",
  "#E6BEFF",
  "#0000FF",
  "#FF6347",
  "#FF1493",
  "#E71BE6",
  "#565DFD",
  "#A2F852"
)

cluster<- setNames(colorcode, unique(spe_cd45_9st$immune_9st))
color_vectors <- list()
color_vectors$cluster <- cluster
metadata(spe_cd45_9st)$color_vectors <- color_vectors

In [ ]:
options(repr.plot.width=12, repr.plot.height=10)
p=dittoDimPlot(spe_cd45_9st, var = "immune_9st", 
             reduction.use = "TSNE", size = 4,
             do.label = FALSE) +scale_color_manual(values = metadata(spe_cd45_9st)$color_vectors$cluster)
ggsave(
  "Figure_revised/9st/TSNE_clusters_immune.pdf",
  plot = p,
  width = 11,
  height = 10,
  units = "in",
  dpi = 600
)

In [ ]:

genes <- c("CD3", "CD161",  "CD163", "CD1c")


for (gene in genes) {
  
  p <- dittoDimPlot(spe_cd45_9st, 
                     var = gene, 
                     assay = "exprs_0.5",
                     reduction.use = "TSNE", 
                     size =4, 
                     colors = viridis(100), 
                     do.label = TRUE) +
       scale_color_viridis()


  pdf_filename <- paste0(gene, "_TSNE_plot.pdf")  

  ggsave(pdf_filename, plot = p, width = 11, height = 10, units = "in", dpi = 300)
 
}


## 4.3 heatmap

In [ ]:
celltype_mean <- aggregateAcrossCells(as(spe_cd45_9st, "SingleCellExperiment"),  
                     ids = spe_cd45_9st$immune_9st, 
                     statistics = "mean",
                     use.assay.type = "exprs_0.5", 
                     subset.row = rownames(spe_cd45_9st)[rowData(spe_cd45_9st)$use_channel])
heatmap_data <- assay(celltype_mean, "exprs_0.5")
head(heatmap_data)

In [ ]:
color_dict <- c(
"0" = "#C1A029",
  "1" = "#ff002a",
  "2" = "#00FFFF",
  "3" = "#FEEC37",
  "4" = "#4fa9ff",
  "5" = "#009203",
  "6" = "#E6BEFF",
  "7" = "#0000FF",
  "8" = "#FF6347",
  "9" = "#FF1493",
  "10" = "#E71BE6",
  "11" = "#565DFD",
  "12" = "#A2F852",
  "13"="#2b2b2b"
)

group <- colnames(heatmap_data)


col_annot <- HeatmapAnnotation(
  Group = group,
  col = list(Group = color_dict)
)

In [ ]:
options(repr.plot.width=8, repr.plot.height=10)
pdf("heatmap_spe_CD45_ComplexHeatmap_zscore_manhattan.pdf", width = 10, height = 10)
z_score_matrix <- t(scale(t(heatmap_data), center = TRUE, scale = TRUE))
col_fun <- colorRamp2(c(-2, 0, 2), c("blue", "white", "red"))
Heatmap(z_score_matrix,
        name = "Z-score",
        col = col_fun,
        top_annotation = col_annot,
        show_row_names = TRUE,
        show_column_names = TRUE,
        clustering_distance_rows = "euclidean",
        clustering_distance_columns = "pearson",  #manhattan   euclidean
        #clustering_method_rows = FALSE,
        clustering_method_columns = "complete",
        row_title = "Genes",
        column_title = "Sample Groups")
dev.off()

## 4.4 quantification

In [ ]:
meta_follicle<- meta%>%filter(follicle_type_updata1 != "1")


percentage_data <- meta_follicle_immune %>%
  group_by(follicle_type_updata1, immune_9st1) %>%
  summarise(count = n(), .groups = "drop") %>%
  group_by(follicle_type_updata1) %>%
  mutate(percentage = (count / sum(count)) * 100) %>%
  ungroup() %>%
  mutate(
    immune_9st1 = factor(immune_9st1, levels = new_levels), 
    follicle_type_updata1 = factor(follicle_type_updata1, levels = c("healthy", "type1", "type2", "type3", "CA","CAlate")) 
  )

In [ ]:
library(ggplot2)
library(dplyr)


summary_df <- result_df %>%
  group_by(follicle_type_updata1) %>%
  summarise(
    mean_percent = mean(percent, na.rm = TRUE),  
    se = sd(percent, na.rm = TRUE) / sqrt(n())   
  )  %>%
mutate(
     follicle_type_updata1= factor(follicle_type_updata1, levels = c("healthy", "type1", "type2", "type3", "CA", "CAlate")) 
    )


p=ggplot(summary_df, aes(x = follicle_type_updata1, y = mean_percent)) +
  geom_bar(stat = "identity", fill = "#E71BE6", width = 0.8) +  
  geom_errorbar(aes(ymin = mean_percent - se, ymax = mean_percent + se), 
                width = 0.2, color = "black") + 
  scale_y_continuous(expand = c(0, 0)) +  
  labs(title = "Percentage of ROI Groups Across Follicle Types",
       x = "Follicle Type",
       y = "Percentage (%)") +
  theme_minimal() +
  theme(
   axis.text.x = element_text(angle = 90,size = 20, face = "bold", color = "black"), 
    axis.text.y = element_text(size = 20, face = "bold", color = "black"),  
    axis.title.x = element_text(face = "bold", size = 20, margin = margin(t = 10)),
    axis.title.y = element_text(face = "bold", size = 20, margin = margin(r = 10)),
    plot.title = element_text(hjust = 0.5, face = "bold", size = 15),
   panel.grid.major = element_blank(), 
    panel.grid.minor = element_blank(),  
    panel.grid.major.y = element_blank(),
    panel.border = element_blank(),
    axis.line = element_line(size = 1, color = "black")
  )
p
ggsave("Figure_revised/9st/percetange_immune_follicle_type_merge.pdf", plot = p, width = 5, height = 10)

In [ ]:
meta_follicle<- meta%>%filter(follicle_type_updata1 != "1")
meta_follicle_immune<- meta_follicle %>%filter(immune_9st1 != "30")
new_levels <- as.character(c(3, 7, 11, 10, 5, 2, 6, 4, 0, 1, 9, 12, 8))

percentage_data <- meta_follicle_immune %>%
  group_by(follicle_type_updata1, immune_9st1) %>%
  summarise(count = n(), .groups = "drop") %>%
  group_by(follicle_type_updata1) %>%
  mutate(percentage = (count / sum(count)) * 100) %>%
  ungroup() %>%
  mutate(
    immune_9st1 = factor(immune_9st1, levels = new_levels),  
    follicle_type_updata1 = factor(follicle_type_updata1, levels = c("healthy", "type1", "type2", "type3", "CA","CAlate"))
  )

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)


p=ggplot(percentage_data, aes(x = follicle_merge, y = percentage, fill =immune_9st)) +
  geom_bar(stat = "identity", position = "stack") +
  scale_fill_manual(values = color_dict) +
  labs(
    title = "Percentage of leiden_1.5_updata in ROI",
    x = "ROI",
    y = "Percentage (%)",
    fill = "Leiden 1.5"
  ) +
  theme_minimal() +
  theme(
    axis.title = element_text(size = 20, face = "bold",color = "black"), 
    axis.text.x = element_text(angle = 90,size = 20, face = "bold", color = "black"),  
    axis.text.y = element_text(size = 20, face = "bold", color = "black"),  
    axis.line = element_line(color = "black", size = 0.8), 
    panel.grid = element_blank() ）
  )
p
ggsave("figure_revised_9st/immune_follicletype_percentage_heatmap_order.pdf", plot = p, width = 10, height = 10

## 4.5 mapping of cell cluster

In [ ]:
color_mapping <- c(
  "1" = "#C1A029",
  "2" = "#ff002a",
  "3" = "#009203",
  "4" = "#FEEC37",
  "5" = "#4fa9ff",
  "6" = "#00FFFF",
  "7" = "#E6BEFF",
  "8" = "#0000FF",
  "9" = "#FF6347",
  "10" = "#FF1493",
  "11" = "#E71BE6",
  "12" = "#565DFD",
  "13" = "#A2F852",
  "30" = "#2b2b2b"
)

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)

plotCells(masks,
          object = spe, 
          cell_id = "ObjectNumber", 
          img_id = "sample_id",
          colour_by = "immune_9st",
          colour = list(immune_9st = color_mapping),
           missing_colour = "#2b2b2b",
           display = "single",
           scale_bar=NULL,
          image_title=NULL,
          save_plot = list(filename = "Figure_revised/9st/mapping for immune/whole_sample_immune.png", scale = 3))

In [ ]:
plotPixels(image =images,
           mask =masks,
           object = spe, 
           cell_id = "ObjectNumber", 
           img_id = "sample_id",
           colour_by =c("CD163"),
           outline_by = "cl8",
           colour = list(cl8= c("1"="#FFFFFF","2"="#45095C")),
            bcg = list(CD163 = c(0, 15, 1)),
          display = "single",
           scale_bar=NULL,
           image_title=NULL,
           image_title=NULL,
           thick = TRUE,
           save_plot = list(filename = "Figure_revised/9st/CD163_cl8.png", scale = 2))